In [ ]:
# Original neural-style documentation: https://github.com/jcjohnson/neural-style

import subprocess, os, curses
from PIL import Image
from IPython.display import display

script_style = ['/usr/local/bin/th', 'neural_style_dir_rng_fix.lua']
script_lincolor = ['python3', 'Neural-Tools/linear-color-transfer.py']
script_lum = ['python3', 'Neural-Tools/lum-transfer.py']

model_vgg = 'models/VGG_ILSVRC_19_layers.caffemodel'
model_vggnorm = 'models/vgg_normalised.caffemodel'
model_nin = 'models/nin_imagenet_conv.caffemodel'
model_nyud = 'models/nyud-fcn32s-color-heavy.caffemodel'
model_channelpruning = 'models/channel_pruning.caffemodel'

def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(str(output.strip(), 'utf-8'))
    rc = process.poll()
    return rc

def style_transfer(style_image,
                   content_image,
                   image_size=None, #defaults to 512
                   style_blend_weights=None, #defaults to equal weights
                   gpu=None, #defaults to 0, use -1 for CPU only (why would you?)
                   content_weight=None, #defaults to '5e0'
                   style_weight=None, #defaults to '1e2'
                   tv_weight=None, #defaults to 1e-3
                   num_iterations=None, #defaults to 1000
                   init=None, #defaults to 'random', pass 'image' to use content_image
                   optimizer=None, #defaults to 'lbfgs', 'adam' is lower mem but worse
                   learning_rate=None, #defaults to '1e1', for ADAM only
                   normalize_gradients=None, #defaults to unset, good for ADAM
                   output_image=None, #defaults to 'out.png'
                   print_iter=None, #defaults to 50, 0 to disable
                   save_iter=None, #defaults to 100, 0 to disable
                   content_layers=None, #defaults to 'relu4_2'
                   style_layers=None, #defaults to 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
                   style_scale=None, #defaults to 1.0
                   original_colors=None, #defaults to 0, 1 for original colors
                   proto_file=None, #defaults to 'models/VGG_ILSVRC_19_layers_deploy.prototxt'
                   model_file=None, #defaults to 'models/VGG_ILSVRC_19_layers.caffemodel'
                   pooling=None, #default is 'max', 'avg' may or may not get better results
                   backend=None, #default is 'nn', 'cudnn' and 'clnn' are other options
                   cudnn_autotune=None, #default is unset, more mem but faster backend
                   multigpu_strategy=None, #default is '', see original documentation
                   lbfgs_num_correction=None, #default is 0, higher gets better results I think
                   seed=None): #default is -1, for random I believe
    
    arg_names = ['style_image','content_image','image_size','style_blend_weights','gpu',
                 'content_weight','style_weight','tv_weight','num_iterations','init',
                 'optimizer','learning_rate','normalize_gradients','output_image',
                 'print_iter','save_iter','content_layers','style_layers','style_scale',
                 'original_colors','proto_file','model_file','pooling','backend',
                 'cudnn_autotune','multigpu_strategy','lbfgs_num_correction','seed']
    
    command = script_style.copy()
    for arg_name in arg_names:
        arg_val = eval(arg_name)
        if arg_val != None:
            arg_val = str(arg_val)
            command.append('-' + arg_name)
            if arg_name not in ['normalize_gradients', 'cudnn_autotune']:
                command.append(arg_val)
    
    #print(command)
    result = run_command(command)

def image_max_size(image_in):
    with Image.open(image_in) as im:
        im_size = im.size
    return max(im_size)

def multires_transfer(style_image,
                      content_image,
                      style_blend_weights=None, #defaults to equal weights
                      gpu=None, #defaults to 0, use -1 for CPU only (why would you?)
                      content_weight=None, #defaults to '5e0'
                      style_weight=None, #defaults to '1e2'
                      tv_weight=None, #defaults to 1e-3
                      init=None, #defaults to 'random', pass 'image' to use content_image
                      optimizer=None, #defaults to 'lbfgs', 'adam' is lower mem but worse
                      learning_rate=None, #defaults to '1e1', for ADAM only
                      normalize_gradients=None, #defaults to unset, good for ADAM
                      output_image=None, #defaults to 'out.png'
                      print_iter=None, #defaults to 50, 0 to disable
                      content_layers=None, #defaults to 'relu4_2'
                      style_layers=None, #defaults to 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
                      style_scale=None, #defaults to 1.0
                      original_colors=None, #defaults to 0, 1 for original colors
                      proto_file=None, #defaults to 'models/VGG_ILSVRC_19_layers_deploy.prototxt'
                      model_file=None, #defaults to 'models/VGG_ILSVRC_19_layers.caffemodel'
                      pooling=None, #default is 'max', 'avg' may or may not get better results
                      multigpu_strategy=None, #default is '', see original documentation
                      lbfgs_num_correction=None, #default is 0, higher gets better results I think
                      
                      steps=5,
                      base_iterations=1000,
                      cuda=True):
    
    final_size = image_max_size(content_image)
    step_size = final_size / (steps+1)
    
    for step in range(steps):
        curr_size = round(step_size * (step+1))
        
        if step == 0:
            img_in = content_image
        else:
            img_in = str(step-1) + '.png'
        
        if step == steps-1:
            img_out = output_image
        else:
            img_out = str(step) + 'png'
        
        iters = round(base_iterations / (step+1))
        iters = max(iters, 20)
        
        if cuda:
            backend = 'cudnn'
            cudnn_autotune = True
        else:
            backend = 'nn'
            cudnn_autotune = None
        
        
        r = style_transfer(style_image=style_image,
                           content_image=img_in,
                           image_size=curr_size, 
                           style_blend_weights=style_blend_weights, 
                           gpu=gpu, 
                           content_weight=content_weight, 
                           style_weight=style_weight, 
                           tv_weight=tv_weight, 
                           num_iterations=iters, 
                           init=init, 
                           optimizer=optimizer, 
                           learning_rate=learning_rate, 
                           normalize_gradients=normalize_gradients, 
                           output_image=img_out, 
                           print_iter=print_iter, 
                           save_iter=0, 
                           content_layers=content_layers, 
                           style_layers=style_layers, 
                           style_scale=style_scale, 
                           original_colors=original_colors, 
                           proto_file=proto_file,
                           model_file=model_file,
                           pooling=pooling,
                           backend=backend,
                           cudnn_autotune=cudnn_autotune,
                           multigpu_strategy=multigpu_strategy,
                           lbfgs_num_correction=lbfgs_num_correction,
                           seed=-1)
        
        
        if os.path.isfile(img_out):
            if img_in != content_image:
                os.remove(img_in)
            if step < steps - 1:
                print('Step {}/{} complete! Proceeding..'.format(step+1, steps))
            else:
                print('All {} steps complete!'.format(steps))
                print('Final image saved to "{}"'.format(output_image))
        else:
            if img_in != content_image:
                os.rename(img_in, output_image)
            else:
                os.copy(img_in, output_image)
            print('Step {}/{} FAILED!'.format(step+1, steps))
            print('Image from previous step saved to "{}"'.format(output_image))
            return


In [ ]:
multires_transfer(style_image = 'test_files/style/',
                  content_image = 'test_files/content.jpg',
                  style_blend_weights = '3,1',
                  style_weight = '5e2',
                  tv_weight = 0,
                  output_image = 'test_files/result.png',
                  print_iter = 1,
                  model_file=model_vgg)

with Image.open('test_files/result.png') as im:
    display(im)